In [3]:
!pip install vit-pytorch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.8/140.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [4]:
import os
import zipfile
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from vit_pytorch import ViT
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error

# Step 1: Clone the GitHub repository containing the dataset
!git clone https://github.com/afeefjunaid/fyp.git  # Replace with your GitHub URL

# Step 2: Define the path to the dataset folder within the cloned repository
dataset_path = '/content/fyp/skinType'  # Adjust this path to your specific dataset directory

# Define hyperparameters
batch_sizes = [16, 32, 64, 128, 256]
learning_rates = [0.001, 0.0001, 0.0005, 0.00001]
optimizers_list = ['SGD', 'Adam', 'RMSprop', 'AdamW']
epochs = 2
num_classes = 3  # Dry, Normal, Oily skin types

# Load Dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Dataset Path
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Split Dataset into Train, Validation, and Test
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Function to initialize the model
def create_vit_model():
    model = ViT(
        image_size=224,
        patch_size=32,
        num_classes=num_classes,  # 3 classes: dry, normal, oily
        dim=64,                   # model embedding dimension
        depth=6,                  # number of transformer layers
        heads=8,                  # number of heads in multi-head attention
        mlp_dim=128,              # hidden dimension in MLP head
        dropout=0.1,
        emb_dropout=0.1
    )
    return model

# Optimizer choices
def get_optimizer(optimizer_name, model_params, lr):
    if optimizer_name == 'SGD':
        return optim.SGD(model_params, lr=lr, momentum=0.9)
    elif optimizer_name == 'Adam':
        return optim.Adam(model_params, lr=lr)
    elif optimizer_name == 'RMSprop':
        return optim.RMSprop(model_params, lr=lr)
    elif optimizer_name == 'AdamW':
        return optim.AdamW(model_params, lr=lr)

# Training Function
def train_model(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    all_preds, all_labels = [], []

    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Record predictions
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return running_loss / len(loader), accuracy

# Validation and Test Function
def evaluate_model(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            # Record predictions
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    try:
        roc_auc = roc_auc_score(all_labels, np.array(all_preds), multi_class='ovr', average='weighted')
    except ValueError:
        roc_auc = 0  # Handle case where ROC AUC cannot be computed
    rmse = np.sqrt(mean_squared_error(all_labels, all_preds))

    return running_loss / len(loader), accuracy, precision, recall, f1, roc_auc, rmse

# Main training loop# Main training loop
def train_and_evaluate(batch_size, lr, optimizer_name, epochs):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Model, Optimizer, Criterion
    model = create_vit_model().to(device)
    optimizer = get_optimizer(optimizer_name, model.parameters(), lr)
    criterion = nn.CrossEntropyLoss()

    # Training
    for epoch in range(epochs):
        train_loss, train_acc = train_model(model, train_loader, optimizer, criterion, device)

        # Unpack all 7 values from the validation evaluation
        val_loss, val_acc, val_precision, val_recall, val_f1, val_roc_auc, val_rmse = evaluate_model(model, val_loader, criterion, device)

        if (epoch + 1) % 10 == 0:  # Print every 10 epochs
            print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                  f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Precision: {val_precision:.4f}, "
                  f"Recall: {val_recall:.4f}, F1: {val_f1:.4f}, ROC AUC: {val_roc_auc:.4f}, RMSE: {val_rmse:.4f}")

    # Final testing after training
    test_loss, test_acc, precision, recall, f1, roc_auc, rmse = evaluate_model(model, test_loader, criterion, device)
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}, ROC AUC: {roc_auc:.4f}, RMSE: {rmse:.4f}")
    return test_acc, precision, recall, f1, roc_auc, rmse


# Grid search across all parameters
for batch_size in batch_sizes:
    for lr in learning_rates:
        for optimizer_name in optimizers_list:
            print(f"\nTraining with Batch Size: {batch_size}, Learning Rate: {lr}, Optimizer: {optimizer_name}")
            test_metrics = train_and_evaluate(batch_size, lr, optimizer_name, epochs)
            print(f"Final Test Metrics with Batch Size {batch_size}, LR {lr}, Optimizer {optimizer_name}: {test_metrics}")


Cloning into 'fyp'...
remote: Enumerating objects: 2737, done.
remote: Total 2737 (delta 0), reused 0 (delta 0), pack-reused 2737 (from 1)
Receiving objects: 100% (2737/2737), 107.49 MiB | 18.03 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Updating files: 100% (3826/3826), done.

Training with Batch Size: 16, Learning Rate: 0.001, Optimizer: SGD


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test Loss: 1.0933, Test Accuracy: 0.3548
Precision: 0.3860, Recall: 0.3548, F1-score: 0.2915, ROC AUC: 0.0000, RMSE: 0.9842
Final Test Metrics with Batch Size 16, LR 0.001, Optimizer SGD: (0.35478260869565215, 0.385967806680175, 0.35478260869565215, 0.2914627532251992, 0, np.float64(0.9842233751409855))

Training with Batch Size: 16, Learning Rate: 0.001, Optimizer: Adam


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test Loss: 1.0794, Test Accuracy: 0.3861
Precision: 0.2795, Recall: 0.3861, F1-score: 0.3242, ROC AUC: 0.0000, RMSE: 1.0052
Final Test Metrics with Batch Size 16, LR 0.001, Optimizer Adam: (0.38608695652173913, 0.27948958882959507, 0.38608695652173913, 0.3242342725186318, 0, np.float64(1.005203851270326))

Training with Batch Size: 16, Learning Rate: 0.001, Optimizer: RMSprop


...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test Loss: 1.1034, Test Accuracy: 0.3739
Precision: 0.2704, Recall: 0.3739, F1-score: 0.2725, ROC AUC: 0.0000, RMSE: 0.8485
Final Test Metrics with Batch Size 32, LR 0.001, Optimizer SGD: (0.3739130434782609, 0.27040045654516265, 0.3739130434782609, 0.2724841426712838, 0, np.float64(0.848528137423857))

Training with Batch Size: 32, Learning Rate: 0.001, Optimizer: Adam


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test Loss: 1.0841, Test Accuracy: 0.3809
Precision: 0.2754, Recall: 0.3809, F1-score: 0.3197, ROC AUC: 0.0000, RMSE: 1.0078
Final Test Metrics with Batch Size 32, LR 0.001, Optimizer Adam: (0.3808695652173913, 0.2754281962728094, 0.3808695652173913, 0.3196720047936231, 0, np.float64(1.0077957004835074))

Training with Batch Size: 32, Learning Rate: 0.001, Optimizer: RMSprop


KeyboardInterrupt: 